In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as T
import numpy as np
import random
import os
import time
import copy
import pandas as pd

In [11]:
cwd = os.getcwd()

cwd

'c:\\DTU - KID\\6. Semester\\MIA_SD'

In [12]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Using cache found in C:\Users\anton/.cache\torch\hub\pytorch_vision_v0.10.0


In [13]:
print(torch. __version__)
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
# Find out if a GPU is available
use_cuda = torch.cuda.is_available()
use_cuda

2.2.1+cu118
cuda


True

In [14]:
auxiliary_data = cwd + '/images/auxiliary_data_set/lfw'
target_model_data = cwd + '/images/target_model/stable_diffusion_1_4_paper'

# Display image for reference
white_torch = torchvision.io.read_image(auxiliary_data + '00000.jpeg')
print("")
T.ToPILImage()(white_torch)

# Display image for reference
wh = torchvision.io.read_image(target_model_data + 'Queen_Elizabeth_II_0001.jpg')
print("")
T.ToPILImage()(wh)

AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'